# Stripe invoice frozen customer fields demo

Invoice finalization makes certain properties on the invoice immutable (https://docs.stripe.com/invoicing/integration/workflow-transitions#post-finalized). According to the docs, invoice.customer_address, invoice.customer_email, etc. *cannot* be changed after the invoice has been finalized.

In practice, this means if you...

1. Make an invoice with customer_address = foo@test.com
2. Finalize the invoice
3. Edit the customer address to bar@test.com
4. The customer's email on the finalized invoice will still be foo@test.com

This notebook demonstrates that changing customer details after invoice finalization doesn't impact the invoice.

# Step 1: Set up Stripe

In [ ]:
%pip install stripe

In [ ]:
%pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

stripe_sk = os.environ['STRIPE_SECRET_KEY']
print('hello world')

In [5]:
import stripe
stripe.api_key = stripe_sk

# Step 2: Create a customer

In [11]:
customer = stripe.Customer.create(
    email="test@email.com",
    description="Customer for testing",
    name="Stripe Customer",
    tax_id_data=[{"type": "eu_vat", "value": "CZ1234567890"}],
    address={
        "city": "San Francisco",
        "state": "California",
        "country": "US"
    }
)

In [12]:
customer_id = customer.id
print(customer_id)

cus_Q8Jd52xGH6DFhR


# Step 3: Create and finalize an invoice

In [13]:
invoice = stripe.Invoice.create(
    customer=customer_id
)

In [14]:
invoice_id = invoice.id
print(invoice_id)

in_1PI33RGC4ECGOX3CNUrWNdGT


In [ ]:
product = stripe.Product.create(name="sour patch kids")
price = stripe.Price.create(product=product.id, unit_amount=1000, currency='usd')
stripe.InvoiceItem.create(
  customer=customer_id,
  price=price.id,
  invoice=invoice_id,
)
stripe.Invoice.finalize_invoice(invoice_id)

In [19]:
invoice = stripe.Invoice.retrieve(invoice_id)
print("Email: ", invoice.customer_email)
print("Address: ", invoice.customer_address)
print("Name: ", invoice.customer_name)
print("Tax IDs: ", invoice.customer_tax_ids)

Email:  test@email.com
Address:  {
  "city": "San Francisco",
  "country": "US",
  "line1": null,
  "line2": null,
  "postal_code": null,
  "state": "California"
}
Name:  Stripe Customer
Tax IDs:  [<CustomerTaxId at 0x79f140904c20> JSON: {
  "type": "eu_vat",
  "value": "CZ1234567890"
}]


# Step 4: Update the customer and re-retrieve invoice

In [ ]:
stripe.Customer.modify(
    id=customer_id,
    email="taylorswift13@email.com",
    name="New Name",
    address={
        "city": "Seattle",
        "state": "Washington",
        "country": "US"
    }
  )

In [ ]:
stripe.TaxId.create(type="eu_vat", value="ATU12345678", owner={"type": "customer", "customer": customer_id})

In [27]:
customer = stripe.Customer.retrieve(customer_id, expand=['tax_ids'])
print("Email: ", customer.email)
print("Address: ", customer.address)
print("Name: ", customer.name)
print("Tax IDs: ", customer.tax_ids)

Email:  taylorswift13@email.com
Address:  {
  "city": "Seattle",
  "country": "US",
  "line1": null,
  "line2": null,
  "postal_code": null,
  "state": "Washington"
}
Name:  New Name
Tax IDs:  {
  "data": [
    {
      "country": "CZ",
      "created": 1716099213,
      "customer": "cus_Q8Jd52xGH6DFhR",
      "id": "txi_1PI2zxGC4ECGOX3Cr4e2dMNp",
      "livemode": false,
      "object": "tax_id",
      "owner": {
        "customer": "cus_Q8Jd52xGH6DFhR",
        "type": "customer"
      },
      "type": "eu_vat",
      "value": "CZ1234567890",
      "verification": {
        "status": "verified",
        "verified_address": "123 TEST STREET",
        "verified_name": "TEST"
      }
    },
    {
      "country": "AT",
      "created": 1716100462,
      "customer": "cus_Q8Jd52xGH6DFhR",
      "id": "txi_1PI3K6GC4ECGOX3CpCtDdHC2",
      "livemode": false,
      "object": "tax_id",
      "owner": {
        "customer": "cus_Q8Jd52xGH6DFhR",
        "type": "customer"
      },
      "type": 

In [28]:
# Invoice values frozen from the customer before finalization
# remain unchanged.
invoice = stripe.Invoice.retrieve(invoice_id)
print("Email: ", invoice.customer_email)
print("Address: ", invoice.customer_address)
print("Name: ", invoice.customer_name)
print("Tax IDs: ", invoice.customer_tax_ids)

Email:  test@email.com
Address:  {
  "city": "San Francisco",
  "country": "US",
  "line1": null,
  "line2": null,
  "postal_code": null,
  "state": "California"
}
Name:  Stripe Customer
Tax IDs:  [<CustomerTaxId at 0x79f142bb0fe0> JSON: {
  "type": "eu_vat",
  "value": "CZ1234567890"
}]
